In [22]:
import os
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

# Set the Groq API key
os.environ["GROQ_API_KEY"] = "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"
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise EnvironmentError("GROQ_API_KEY environment variable is not set.")

# Path to the PDF documents
datasource_paths = {
    "bharatiya nyayaa document": r"C:\Users\joshu\Downloads\NyayMitra-SIH-main\NyayMitra-SIH-main\Source_material\bharatiyanyayasanhita.pdf",
    "code of civil procedure": r"C:\Users\joshu\Downloads\NyayMitra-SIH-main\NyayMitra-SIH-main\Source_material\codeofcivilprocedure.pdf",
    "Indian contact act 1872": r"C:\Users\joshu\Downloads\NyayMitra-SIH-main\NyayMitra-SIH-main\Source_material\Indiancontactact1872.pdf",
    "Specific Relief act": r"C:\Users\joshu\Downloads\NyayMitra-SIH-main\NyayMitra-SIH-main\Source_material\specific_relief_act.pdf",
    "Transfer of property act": r"C:\Users\joshu\Downloads\NyayMitra-SIH-main\NyayMitra-SIH-main\Source_material\transferofpropertyact.pdf",
    "Indian constitution": r"C:\Users\joshu\Downloads\NyayMitra-SIH-main\NyayMitra-SIH-main\Source_material\indianconstitution.pdf"
}

class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""
    datasource: Literal[
        "bharatiya nyayaa document", 
        "code of civil procedure", 
        "Indian contact act 1872", 
        "Specific Relief act", 
        "Transfer of property act", 
        "Indian constitution"
    ] = Field(
        ...,
        description="Given a user question, choose which datasource would be most relevant for answering their question",
    )

    class Config:
        arbitrary_types_allowed = True

# Initialize the language model
llm = ChatGroq(temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Define the system prompt
system = """You are an expert at routing a user question to the appropriate data source.
Based on the content or legal context the question is referring to, route it to the relevant data source."""

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

# Function to get the correct document path based on the routed datasource
def get_document_path(routed_result):
    datasource = routed_result.datasource
    return datasource_paths.get(datasource, "Datasource not found")

# Invoke the router with a sample question
result = router.invoke("What is the specific relief act?")
document_path = get_document_path(result)

# Output the result and the linked document path
print(f"Routed to: {result.datasource}")
print(f"Linked Document Path: {document_path}")


Routed to: Specific Relief act
Linked Document Path: C:\Users\joshu\Downloads\NyayMitra-SIH-main\NyayMitra-SIH-main\Source_material\specific_relief_act.pdf


In [28]:
import os
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Judiciary assistant prompts
nyayaa_template = """You are an expert in the Bharatiya Nyaya Sanhita. \
You provide clear and concise answers related to this document. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

civil_procedure_template = """You are an expert in the Code of Civil Procedure. \
You provide detailed and accurate answers related to this document. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""
contract_act_template = """You are an expert in the Indian Contract Act 1872. \
You provide precise answers regarding contracts and agreements as per this law. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

specific_relief_template = """You are an expert in the Specific Relief Act. \
You provide concise and clear answers regarding specific relief under Indian law. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

property_act_template = """You are an expert in the Transfer of Property Act. \
You give well-defined answers regarding the transfer of property and ownership in India. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

constitution_template = """You are an expert in the Indian Constitution. \
You provide answers about fundamental rights, duties, and governance under this law. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

# Default prompt for general judiciary-related questions
default_judiciary_template = """You are an AI assistant specializing in judiciary matters in India. \
You provide general legal information and help users navigate through the various Indian legal documents. \
When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

# Embed prompts
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
prompt_templates = [
    nyayaa_template, civil_procedure_template, contract_act_template,
    specific_relief_template, property_act_template, constitution_template,
    default_judiciary_template
]
prompt_embeddings = hf_embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = hf_embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    if most_similar == nyayaa_template:
        print("Using Bharatiya Nyaya Sanhita Template")
    elif most_similar == civil_procedure_template:
        print("Using Code of Civil Procedure Template")
    elif most_similar == contract_act_template:
        print("Using Indian Contract Act Template")
    elif most_similar == specific_relief_template:
        print("Using Specific Relief Act Template")
    elif most_similar == property_act_template:
        print("Using Transfer of Property Act Template")
    elif most_similar == constitution_template:
        print("Using Indian Constitution Template")
    else:
        print("Using Default Judiciary Template")
    
    # Return the chosen prompt
    return PromptTemplate.from_template(most_similar)

# Define the chain
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatGroq()
    | StrOutputParser()
)

# Test the chain with a sample question
print(chain.invoke("What is the 4th part under the constitution?"))


Using Indian Constitution Template
The Indian Constitution is divided into 22 Parts. Therefore, there is no Fourth Part under the Constitution. The Fourteenth Part of the Constitution deals with “Temporary, Transitional and Special Provisions.” If you meant to ask about a specific fundamental right or duty that is the fourth in the list, I would need more information as the Fundamental Rights are guaranteed under Part III of the Constitution and there are six of them. The Fundamental Duties were added by the 42nd Amendment in 1976 and there are ten of them.


In [33]:
import datetime
from typing import Literal, Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class LegalDocumentSearch(BaseModel):
    """Search over a database of legal documents."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to document content.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to document titles. "
            "Should be succinct and only include key words that could be in a document title."
        ),
    )
    min_page_count: Optional[int] = Field(
        None,
        description="Minimum page count filter, inclusive. Only use if explicitly specified.",
    )
    max_page_count: Optional[int] = Field(
        None,
        description="Maximum page count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_word_count: Optional[int] = Field(
        None,
        description="Minimum word count filter, inclusive. Only use if explicitly specified.",
    )
    max_word_count: Optional[int] = Field(
        None,
        description="Maximum word count filter, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        """Print the search criteria in a readable format."""
        for field in self.__fields__:
            value = getattr(self, field)
            if value is not None and value != getattr(self.__fields__[field], "default", None):
                print(f"{field}: {value}")

# Example usage
search_criteria = LegalDocumentSearch(
    content_search="indian contact act",
    title_search="indian contact act",
    min_page_count=10,
    earliest_publish_date=datetime.date(2000, 1, 1)
)

search_criteria.pretty_print()


content_search: indian contact act
title_search: indian contact act
min_page_count: 10
earliest_publish_date: 2000-01-01


In [34]:
from langchain_core.prompts import ChatPromptTemplate

# System prompt for legal document search
system = """You are an expert at converting user questions into database queries for legal documents. \
You have access to a database of legal documents including laws, acts, and legal codes. \
Given a question, return a database query optimized to retrieve the most relevant legal documents.

If there are legal terms or acronyms you are not familiar with, do not try to rephrase them."""

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Initialize the language model
llm = ChatGroq()  # Ensure you have the appropriate model initialized
structured_llm = llm.with_structured_output(LegalDocumentSearch)  # Assuming LegalDocumentSearch is defined

# Define the query analyzer
query_analyzer = prompt | structured_llm


In [35]:
# Sample user question
user_question = "What are the key provisions of the Indian Contract Act?"

# Analyze the query using the query analyzer
query_result = query_analyzer.invoke(user_question)

# Print the structured search criteria
print(query_result)


content_search='key provisions' title_search='Indian Contract Act' min_page_count=None max_page_count=None earliest_publish_date=None latest_publish_date=None min_word_count=None max_word_count=None
